In [ ]:
import tensorflow as tf
import numpy as np
import keras
import PIL

In [ ]:
# Load model
classif_model = keras.models.load_model("kanotype_deepdanbooru_resnet-custom-v2.hdf5")

# Load tags
with open("classif/danbooru-resnet_custom_v2-p4/tags.txt", 'r') as tags_stream:
    tags = np.array([tag for tag in (tag.strip() for tag in tags_stream) if tag])

In [ ]:
# Load image
in_path = "download20190203202101.png"
image = np.array(PIL.Image.open(in_path).convert('RGB').resize((299, 299))) / 255.0

# Decode
results = classif_model.predict(np.array([image])).reshape(tags.shape[0])

In [ ]:
# Threshold and get tags
threshold = 0.1
result_tags = {}
for i in range(len(tags)):
    if results[i] > threshold:
        result_tags[tags[i]] = results[i]

# Print in order        
sorted_tags = reversed(sorted(result_tags.keys(), key = lambda x: result_tags[x]))
for tag in sorted_tags:
    print('{0: <32} {1:04f}'.format(tag, result_tags[tag]))

In [ ]:
perceptual_model = keras.Model(classif_model.input, classif_model.layers[-5].output)

in_paths = [
    "download20190203202101.png",
    "download20190203202101_2.png",
    "download20190203201352.png",
    "download20190206162412.png",
    "download20190206190813.png",
    "download20190203203134.png"
]

image_results = []
for in_path in in_paths:
    image = np.array(PIL.Image.open(in_path).convert('RGB').resize((299, 299))) / 255.0
    image_results.append(perceptual_model.predict(np.array([image])).flatten())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

matrix = []
for y in range(6):
    for x in range(6):
        matrix.append(np.mean(np.square(image_results[x] - image_results[y])))
matrix = np.array(matrix).reshape(6, 6)
plt.imshow(matrix)